# One-way  ANOVA
## 분산분석의 이해
두 집단의 평균을 비교할 때는 t 검정을 쓰고, 3개 이상의 집단의 평균을 비교할 때는 분산 분석을 수행한다.

두 집단의 경우에 분산분석을 해도 되는데, 그때는 t 검정과 결과가 동일함

세 개 이상의 집단을 대상으로 두 집단씩 여러번 t검정을 수행하게 되면 신뢰도가 급격히 감소하게 되는데 분산 분석은 이러한 현상이 발생하지 않는다.

분산
- 데이터의 변동 정도를 의미하는 지표
- 편차의 제곱에 대한 평균
- 분산이 크다 --> 데이터의 산포도가 크다 (데이터가 넓게 펴져있음)

분산분석의 조건

| 조건| 설명 |
|--|--|
| 정규성| 각 샘플 데이터는 정규분포를 따르는 모집단으로 부터 추출되었다 |
| 등분산성 |Y의 모집단 분산은 각각의 모집단에서 동일 |
| 독립성 | 모집단 간의 오차는 서로 독립이어야함, 각 그룹내의 관측치들이 서로 독립이어야함 |


분산 분석의 종류

|종류 |	이름 |	설명 |
| -- | -- | -- |
| 일원분산분석 | One-way ANOVA	|종속변수1개, 독립변수 집단도 1개인 경우 , 한가지 변수의 변화가 결과 변수에 미치는 영향을 알아보기 위해서 사용됨|
| 이원분산분석 | Two-way ANOVA|	독립변수 집단이 두 개 이상인 집단간 차이가 유의한지를 검증하기 위해 사용 ,한 변수의 변화가 결과에 미치는 영향이 다른 변수의 수준에 따라 달라지는지를 확인하기 위해 사용 |

F검정의 가설
| 가설 | 설명|
|--|--|
|귀무가설|	모든 집단의 평균이 같다	|
|대립가설|	어떤 집단의 평균이 다르다	|

p < 유의수준이면 귀무가설을 기각하고, 대립가설을 채택한다.

대립가설을 자세히 보시면 "어떤 집단의 평균이 다르다"이다.

집단이 A, B, C가 있으면 A와 B가 다른지 B와 C가 다른지는 분산 분석으로 알 수 없다.

> A와 B가 다른지 B와 C가 다른지는 사후 검정으로 확인해야 한다.

### 파이썬에서 ANOVA를 사용하기 위한 패키지

> scipy.stats 패키지와 statsmodels 패키지가 ANOVA 검정 방법을 제공한다.


In [1]:
import pandas as pd
from scipy.stats import shapiro, normaltest, ks_2samp, bartlett, fligner,levene, f_oneway, chi2_contingency
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import sys
import os
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))
from helper import equal_variance_test , normality_test, independence_test,all_test


#### 데이터 가져오기

어떤 지역에서 동일한 제품을 판매하고 있는 두 가게에 대한 일별 매출액


In [2]:
df = pd.read_excel("https://data.hossam.kr/E02/store.xlsx")
df

,store1,store2
0,46,78
1,47,57
2,58,31
3,47,28
4,27,67
5,58,77
6,56,36
7,26,57
8,47,36
9,25,57


## #02. 분산분석의 조건 충족 여부 검사
### 1. 데이터의 정규성 검성
#### 1) shapiro wilks 검정
샘플수가 적을 떄 정규성을 확인하는 검정

(샘플의 수가 대략 50개 미만인 경우, 중심극한 정리는 30개 미만을 권장하기도 함)

| 가설 | 내용 |
|--|--|
| 귀무가설 | 집단간 데이터 분포에는 차이가 없다 (정규분포 따른다) | 
| 대립가설 | 집단간 데이터 분포에는 차이가 있다 (정규분포 따르지 않음) | 



In [3]:
shapiro(df['store1'])
## 정규성 따르지 않음

ShapiroResult(statistic=0.8321117162704468, pvalue=0.035477906465530396)

In [4]:
shapiro(df['store2'])
## 정규성 따름

ShapiroResult(statistic=0.8993193507194519, pvalue=0.21535511314868927)

#### 2) Normal Test

In [5]:
normaltest(df['store1'])
## 정규성 따름

c:\Users\hamin\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


NormaltestResult(statistic=1.9891717867491527, pvalue=0.369876581177278)

In [6]:
normaltest(df['store2'])
## 정규성 따름

NormaltestResult(statistic=2.081349912435389, pvalue=0.3532161960163575)

#### 3) 콜모고로프 - 스미로프 검정

정규분포에 국한되지않고, 두 표본이 같은 분포를 따르는지 확인할 수 있다.

한번에 두개씩 검사 가능

In [7]:
ks_2samp(df['store1'],df['store2'])
## 두 집단의 분포가 같음

KstestResult(statistic=0.4, pvalue=0.41752365281777043, statistic_location=56, statistic_sign=1)

### 2. 데이터의 등분산성 검정
#### 1) Bartlett 검정

집단간 분산이 같은지 다른지 여부를 알아볼 때 사용

독립2표본 t검정 또는 일원분산분석 실시전에 등분산성을 확인하는 용도

Bartlett 검정은 두 집단 뿐만 아니라 세 집단 이상에서도 사용할 수 있음.

모든 변수가 정규분포를 따른다는 가정 하에서만 사용가능함

| 가설 | 내용 |
|--|--|
| 귀무가설 | 집단간 분산의 차이가 없다  | 
| 대립가설 | 집단간 분산의 차이가 있다  | 


In [8]:
bartlett(df['store1'],df['store2'])
## 등분산 성립

BartlettResult(statistic=1.0488412011085946, pvalue=0.305774119649436)

### 2) fligner 검정

Filgner-Killeen test는 비모수 등분산 검정으로 각 독립 표본들이 정규분포를 따르지 않아도 사용한 검정 방법

| 가설 | 내용 |
|--|--|
| 귀무가설 | 집단간 분산의 차이가 없다  | 
| 대립가설 | 집단간 분산의 차이가 있다  | 

In [9]:
fligner(df['store1'],df['store2'])

## 등분산 성립

FlignerResult(statistic=1.3014081560908837, pvalue=0.2539561678380817)

#### 3) Levene 검정

통계학에서 등분산성을 검증하기 위해 사용되는 방법

분석하려는 데이터 그룹이 두개 이상인 경우 

다른 등분산성 검정 방법과 달리 레빈 검정은 정규성 가정을 거의 하지 않기 떄문에 비모수적인 방법으로도 적용할 수 있다.

| 가설 | 내용 |
|--|--|
| 귀무가설 | 집단간 분산의 차이가 없다  | 
| 대립가설 | 집단간 분산의 차이가 있다  | 


In [10]:
levene(df['store1'],df['store2'])
## 등분산 성립

LeveneResult(statistic=1.333315753388535, pvalue=0.2633161881599037)

#### 함수로 한번에 처리

In [11]:
equal_variance_test(df['store1'],df['store2'])
## 등분산 성립

Statistic   p-value  Result
Condition Test     Field                                        
등분산성      Bartlett store1 vs store2   1.048841  0.305774    True
          Fligner  store1 vs store2   1.301408  0.253956    True
          Levene   store1 vs store2   1.333316  0.263316    True

In [12]:
chi2_contingency(df[['store1','store2']])
## 각 그룹이 서로 
## 0.05 보다 작으면 두 변수는 서로 독립적이지 않다 0.05보다 커야 독립

Chi2ContingencyResult(statistic=64.44306604494015, pvalue=1.8512336438945882e-10, dof=9, expected_freq=array([[56.38709677, 67.61290323],
       [47.29240375, 56.70759625],
       [40.47138398, 48.52861602],
       [34.10509886, 40.89490114],
       [42.74505723, 51.25494277],
       [61.38917794, 73.61082206],
       [41.83558793, 50.16441207],
       [37.74297607, 45.25702393],
       [37.74297607, 45.25702393],
       [37.28824142, 44.71175858]]))

## #04. scipy.stats 패키지를 사용한 분산분석


In [13]:
f_oneway(df['store1'],df['store2'])

F_onewayResult(statistic=1.4591624718860445, pvalue=0.24269553293319623)

## #05. statsmodels 패키지를 사용한 일원분산분석

### 데이터 전처리

In [14]:
df2=df.melt(var_name='store',value_name='sales')
df2

,store,sales
0,store1,46
1,store1,47
2,store1,58
3,store1,47
4,store1,27
5,store1,58
6,store1,56
7,store1,26
8,store1,47
9,store1,25


### 범주형으로 바꿔준다 

In [15]:
df2['store'] = df2['store'].astype("category")

df2['store'].dtypes

CategoricalDtype(categories=['store1', 'store2'], ordered=False)

In [16]:
lm = ols("sales ~ C(store)",data=df2).fit()
anova_lm(lm)

,df,sum_sq,mean_sq,F,PR(>F)
C(store),1.0,378.45,378.450000,1.459162,0.242696
Residual,18.0,4668.50,259.361111,NaN,NaN


In [17]:
normality_test(df['store1'],df['store2'])

c:\Users\hamin\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


Statistic   p-value  Result
Condition Test      Field                                        
정규성       Shapiro   store1             0.832112  0.035478   False
                    store2             0.899319  0.215355    True
          normal    store1             1.989172  0.369877    True
                    store2             2.081350  0.353216    True
          k-s_2samp store1 vs store2   0.400000  0.417524    True
                    store2 vs store1   0.400000  0.417524    True

In [18]:
independence_test(df['store1'],df['store2'])

,,,Statistic,p-value,Result
Condition,Test,Field,,,
독립성,Chi2,store1 vs store2,64.443066,1.851234e-10,False


In [19]:
all_test(df['store1'],df['store2'])

c:\Users\hamin\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


Statistic       p-value  Result
Condition Test      Field                                            
정규성       Shapiro   store1             0.832112  3.547791e-02   False
                    store2             0.899319  2.153551e-01    True
          normal    store1             1.989172  3.698766e-01    True
                    store2             2.081350  3.532162e-01    True
          k-s_2samp store1 vs store2   0.400000  4.175237e-01    True
                    store2 vs store1   0.400000  4.175237e-01    True
등분산성      Bartlett  store1 vs store2   1.048841  3.057741e-01    True
          Fligner   store1 vs store2   1.301408  2.539562e-01    True
          Levene    store1 vs store2   1.333316  2.633162e-01    True
독립성       Chi2      store1 vs store2  64.443066  1.851234e-10   False